In [1]:
import os
import glob
import json
from os import path

import sys
sys.path.append("/home/shtoshni/Research/events/src")

from kbp_2015_utils.constants import SPLIT_TO_DIR, SUBDIR_DICT, SUBDIR_EXT
from data_processing.kbp_2015.utils import parse_ann_file

In [2]:
base_input_dir = "/home/shtoshni/Research/events/data/kbp_2014-2015/data/2015"
output_dir = "/home/shtoshni/Research/events/proc_data/kbp_2015/remove_xml"

### Function to load tokens in StanfordNLP processed JSON file

In [9]:
def load_tokens_in_json(json_file):
#     test_file = "/home/shtoshni/Research/events/proc_data/kbp_2015/remove_xml/3f71fead3fa119ccdcdf01769ffee5b1.txt.json"
    data = json.loads(open(json_file).read())
    sentences = data["sentences"]
#     print(len(sentences))

    token_idx_to_char = dict()

    for sentence in sentences:
        tokens = sentence["tokens"]
        speaker_set = set()
        for token in tokens:
            if "speaker" in token:
                speaker_set.add(token["speaker"])
            
            offset = token["characterOffsetBegin"]
            token_text = token["originalText"]
            for token_idx in range(token["characterOffsetBegin"], token["characterOffsetEnd"]):
                token_idx_to_char[token_idx] = token_text[token_idx - offset]
        
        assert (len(speaker_set) <= 1)
        
    return token_idx_to_char
#     print(len(token_start_dict))

### Verify files

#### Verification reveals that only 3 of the annotations marked in URLs are ignored in preprocessing. That's fine!

In [10]:
def verify_partition(split):
    split_dir = path.join(base_input_dir, SPLIT_TO_DIR[split])
    source_dir = path.join(split_dir, SUBDIR_DICT["source"])
    ann_dir = path.join(split_dir, SUBDIR_DICT["ann"])

    source_files = sorted(glob.glob(path.join(source_dir, "*" + SUBDIR_EXT["source"])))
    
    proc_source_files, ann_files = [], []
    for source_file in source_files:
        source_text = open(source_file).read()
        doc_id = path.splitext(path.basename(source_file))[0]
        ann_file = path.join(ann_dir, doc_id + SUBDIR_EXT["ann"])
        proc_source_file = path.join(output_dir, path.basename(source_file) + ".json")            
        assert (path.exists(ann_file))
        assert (path.exists(proc_source_file))
        
        ann_files.append(ann_file)
        proc_source_files.append(proc_source_file)
        
        token_idx_to_char = load_tokens_in_json(proc_source_file)
        mention_list = parse_ann_file(ann_file)[1]
        
        for (span_start, span_end, _) in mention_list:
            if (span_start not in token_idx_to_char) or ((span_end - 1) not in token_idx_to_char):
                print(span_start, span_end -1, source_text[span_start-30:span_end+30], source_file,)
               
#     print(proc_source_files[0], ann_files[0])

for split in SPLIT_TO_DIR:
    verify_partition(split)

562 568 raph.com.au/news/national/men-charded-over-sharia-law-lashing/story /home/shtoshni/Research/events/data/kbp_2014-2015/data/2015/mod_training/source/bolt-eng-DF-170-181109-47916.txt
586 592 l/men-charded-over-sharia-law-lashing/story-e6freuzr-1226097641832" /home/shtoshni/Research/events/data/kbp_2014-2015/data/2015/mod_training/source/bolt-eng-DF-170-181109-47916.txt
242 248 /2010/12/nigerias-drop-cheney-charges-illegal/">Nigeria’s deal to d /home/shtoshni/Research/events/data/kbp_2014-2015/data/2015/mod_dev/source/bolt-eng-DF-170-181109-48534.txt
